# Working with pywikibot, Wikidata API, and SPARQL

In my reconciliation work, my task is usually to take a string, throw that against an API to search, and evaluate the responses to make the linkage between that string to a URI. This is what's meant by the saying "from strings to things."  

A specific use case I've encountered will require a different approach. The use case is: I have already reconciled some strings to [VIAF](https://viaf.org/). I then have a list of VIAF IDs. Wikidata is known to have VIAF IDs within its entities. So how can I reconcile the VIAF IDs to Wikidata? 

There's relatively simple ways to do this assuming I have one VIAF ID. But what if I have thousands? 

This notebook will first explore the `pywikibot` python program to explore the Wikidata API and see how the data is structured and modeled. We'll then see if we can tackle our problem using SPARQL. 

## Setup and Working through the pywikibot tutorial 

Of course we should always start by reading the manual, and helpfully Wikidata has its own ["data harvest" tutorial](https://www.wikidata.org/wiki/Wikidata:Pywikibot_-_Python_3_Tutorial/Data_Harvest) for `pywikibot`. The following section will outline how to explore an item/"page". While this isn't our exact use case, we can start to understand what "claims" are. Claims are important because VIAF IDs, which is what we are after, have a specific claim/property: [P214](https://www.wikidata.org/wiki/Property:P214)

In [1]:
import pywikibot

In [2]:
site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()
item = pywikibot.ItemPage(repo, "Q76")

In [3]:
item_dict = item.get()
clm_dict = item_dict["claims"]
clm_list = clm_dict["P214"]

At this point we could just print the claim value to get a FAST ID. But first we should explore the structure of this specific claim in case we'll need that info later

In [4]:
for clm in clm_list:
    print(clm.toJSON())

{'rank': 'preferred', 'id': 'q76$9AF526A1-C489-4E26-93E0-B831DE7EC2AD', 'mainsnak': {'snaktype': 'value', 'datatype': 'external-id', 'datavalue': {'value': '52010985', 'type': 'string'}, 'property': 'P214'}, 'references': [{'snaks': {'P143': [{'snaktype': 'value', 'datatype': 'wikibase-item', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 8447}, 'type': 'wikibase-entityid'}, 'property': 'P143'}]}, 'hash': 'd4bd87b862b12d99d26e86472d44f26858dee639', 'snaks-order': ['P143']}], 'type': 'statement'}


Now we can just print the VIAF ID for Obama:

In [5]:
for clm in clm_list:
    clm_trgt = clm.getTarget()
    print(clm_trgt)

52010985


And it checks out. The [Wikidata page](https://www.wikidata.org/wiki/Q76) confirms this is the correct VIAF ID. We're starting to see what we might need for our use case. 

## Making a SPARQL query with a VIAF ID

Can we come at this in the opposite way? That is, can we take a VIAF ID and query wikidata via SPARQL? It's actually pretty simple, which should make sense given how SPARQL can handle relation-specific queries (just like SQL).

From the [Wikidata Query Service](https://query.wikidata.org/), we can run the following query:
```
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?person ?personLabel WHERE {
  ?person wdt:P214 "52010985"   
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
  }
}
```

The [result](http://tinyurl.com/llsqors) brings back the correct "Q" wikidata item/page from above, Q76, with the correct label.

We did this using Wikidata's own GUI interface. It's convenient, has autocomplete for its entities and properties, and infers all kinds of things as you type out the query. But we need to figure out how to do this programmatically via python. Let's run the same query using the python library [SPARQLWrapper](https://rdflib.github.io/sparqlwrapper/). Note: It's also perfectly fine to use other libraries, including just good old  

In [3]:
import SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON

In [33]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""    
    SELECT ?person ?personLabel
    WHERE {
  ?person wdt:P214 "52010985"   
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en"
  }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["personLabel"]["value"])
    print(result["person"]["value"])

Barack Obama
http://www.wikidata.org/entity/Q76


This is what we're looking for. We get the label, and the exact URI for Q76 in Wikidata.  

## Iterating a SPARQL query, using a list of values 

We're getting close to solving our original problem. We have successfully queried Wikidata with a VIAF ID and have found the Wikidata URI it matches. But this is so far a one-by-one operation. We need to be able to run this same query, except we want to substitute in all of our VIAF IDs each time, potentially thousands. Knowing the basics of python, you could probably guess we need a `for` loop. And it would make sense to make our list of VIAF IDs into a list that we can then loop over.  

Note: I've included the query `print` so that it's easier to troubleshoot. 

In [4]:
import ujson

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
viaf_id = ["52010985", "34562701", "108815043", "76323201", "2487523", "55588240"]


for f in viaf_id:
    queryString = 'SELECT ?person ?personLabel WHERE { ?person wdt:P214 "' + f + '" SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }}'
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    result1 = sparql.query().convert()
    print(queryString)
    if (len(result1["results"]["bindings"])) == 0:
        print("No match" + '\n')
    for result in result1["results"]["bindings"]:
        print(result["personLabel"]["value"])
        print(result["person"]["value"] + '\n')
                

SELECT ?person ?personLabel WHERE { ?person wdt:P214 "52010985" SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }}
Barack Obama
http://www.wikidata.org/entity/Q76

SELECT ?person ?personLabel WHERE { ?person wdt:P214 "34562701" SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }}
Herbert York
http://www.wikidata.org/entity/Q1609351

SELECT ?person ?personLabel WHERE { ?person wdt:P214 "108815043" SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }}
No match

SELECT ?person ?personLabel WHERE { ?person wdt:P214 "76323201" SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }}
Avrum Stroll
http://www.wikidata.org/entity/Q4829518

SELECT ?person ?personLabel WHERE { ?person wdt:P214 "2487523" SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }}
No match

SELECT ?person ?personLabel WHERE { ?person wdt:P214 "55588240" SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }}
Roger Revelle
http://www.wikidata.org

This looks fine, but notice how newlines (`\n`) had to be hand entered to make the output legible? This would make it hard to work with if you had hundreds of things to reconcile. So let's make this into a data frame with `pandas`. 

In [13]:
import pandas as pd

importantPeople = []
for f in viaf_id:
    queryString = 'SELECT ?person ?personLabel WHERE { ?person wdt:P214 "' + f + '" SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }}'
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    result1 = sparql.query().convert()
    for result in result1["results"]["bindings"]:
        importantPeople.append({
                'Wikidata_URI': result["person"]["value"],            
                'name': result["personLabel"]["value"]
            })
df = pd.DataFrame(importantPeople)
df.head()

,Wikidata_URI,name
0,http://www.wikidata.org/entity/Q76,Barack Obama
1,http://www.wikidata.org/entity/Q1609351,Herbert York
2,http://www.wikidata.org/entity/Q4829518,Avrum Stroll
3,http://www.wikidata.org/entity/Q942808,Roger Revelle


## Iterating over a file 

Now instead of a list of VIAF IDs, let's introduce a more realistic scenario: I have a delimited file (csv,Excel, etc.) with a column of hundreds or even thousands of VIAF IDs. Some VIAD ID cells will be null, as the initial reconciliation process did not have a 100% reconciliation result. One way we could do this is to just treat this new file as a dataframe

Let's first check out and get some help on the `pandas` method `read_csv`: 

In [5]:
help(pd.read_csv)

Help on function read_csv in module pandas.io.parsers:

read_csv(filepath_or_buffer, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=False, error_bad_lines=True, warn_bad_lines=True, skip_footer=0, doublequote=True, delim_whitespace=False, as_recarray=False, compact_ints=False, use_unsigned=False, low_memory=True, buffer_lines=None, memory_map=False, float_precision

In [7]:
viaf_data = 'name_viaf-refine_test.csv'
df1 = pd.read_csv(viaf_data)

df1[0:9]

,AuthoritativeLabel,VIAF_ID,VIAF_URIs,Wikipedia_URLs,Wikidata_IDs
0,UC Regents,NaN,NaN,NaN,NaN
1,Herbert F. York,34562701,https://viaf.org/viaf/34562701,https://en.wikipedia.org/wiki/Herbert_York,"[""Q1609351""]"
2,Andrew H. Wright,108815043,https://viaf.org/viaf/108815043,NaN,NaN
3,Stanley Chodorow,2487523,https://viaf.org/viaf/2487523,NaN,NaN
4,Melvin J. Voight,NaN,NaN,NaN,NaN
5,World Series (Baseball),135197943,https://viaf.org/viaf/135197943,NaN,NaN
6,Hans Seuss,NaN,NaN,NaN,NaN
7,Karen Fleckenstein,NaN,NaN,NaN,NaN
8,Leonard Newmark,92278621,https://viaf.org/viaf/92278621,NaN,NaN


In [10]:
viaf_ids = df1['VIAF_ID'].tolist()
print(viaf_ids)

[nan, '34562701', '108815043', '2487523', nan, '135197943', nan, nan, '92278621', '76323201', nan, '255131945', '27196759', nan, '109530985', nan, '19741752', '245102411', '108378457', '46822320', '235002633', '56672226', '79069733', '44392158', '65331054', '54248007', '45569200', nan, nan, '306193935', '108307703', '94796277', '217920960', '46792221', nan, '55588240', '170016300', '72308491', nan, '27377965', '24685271', '165136992', '117991221', '11963854', '167627219', nan, '65508148', '21443041', nan, nan, nan, '7435849', '33442970', nan, nan, '69008505', '94432205', nan, '229484529', '34777159', '21106583', '192987647', '94924486', nan, '19755215', '234553964', '43586494', '32074777', '119197886', '111308881', '120471026', '26073878', nan, '111883', '47485057', '4075624', '11446679', '8990643', nan, '229956275', nan, '109270251', '101088057', '54237564', '56504103', '51035951', '56709009', '50746062', '161945656', '1384145857091522921764', '299239150', '11336329', '75175060', '109